<a href="https://colab.research.google.com/github/VolkhinD/santiment_analysis/blob/main/sentiment_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
%%capture
!pip install google-play-scraper

In [18]:
import json
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app, reviews_all

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)

From [Google Aps](https://play.google.com/store/apps/) I selected from Top free apps




In [19]:
app_packages = [
  'com.video.editor.filto',
  'com.zhiliaoapp.musically',
  'com.squareup.cash',
  'com.espn.score_center',
  'com.dailymoods.diary'
]

Next using [Google-Play-Scraper](https://github.com/JoMingyu/google-play-scraper) library 

In [21]:
app_infos = []
for ap in tqdm(app_packages):
  info = app(ap, lang='en', country='us')
  del info['comments']
  app_infos.append(info)

100%|██████████| 5/5 [00:01<00:00,  4.82it/s]


In [23]:
def print_json(json_object):
  """ Helper Function that shows data in more beautiful way"""
  json_str = json.dumps(
    json_object,
    indent=2,
    sort_keys=True,
    default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

'histogram' >> actual number of reviews by # of stars

In [24]:
print_json(app_infos[0])

{
  "adSupported": true,
  "appId": "com.video.editor.filto",
  "containsAds": true,
  "contentRating": "Everyone",
  "contentRatingDescription": null,
  "currency": "USD",
  "description": "Best photo/video editor with customized aesthetic sparkle filters & VHS effects which updated weekly. The coolest video & photo editor with multi-style content for creators! It's the easiest to use on IG and Facebook!\r\n\r\nIn Filto, there is a large number of vintage filters, aesthetic sparkle effects,dynamic stickers and other advanced functions. You can trim your video as you need, you are the definition for your creatives! By the way, you can adjust the ratio of your works to match all social media like Instagram, Tik Tok and other social medias, can't wait to see your post attracting numerous likes!\r\n\r\n\ud83c\udf08[Aesthetic FILTERS]\r\nGlITCH: Distort your photos and videos in creative ways\r\nVHS: Full of the fashion with vintage vibe, let\u2019s have party in the 80s~\r\nBLING: Sparkle

In [25]:
app_infos_df = pd.DataFrame(app_infos)
app_infos_df.to_csv('apps.csv', index=None, header=True)

## Get App Reviews

In [26]:
app_reviews = []
for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      rvs, _ = reviews(
        ap,
        lang='en',
        country='us',
        sort=sort_order,
        count= 400 if score == 3 else 200,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)

100%|██████████| 5/5 [00:19<00:00,  3.90s/it]


In [27]:
print_json(app_reviews[325])

{
  "appId": "com.video.editor.filto",
  "at": "2022-09-16 05:41:44",
  "content": "-2 actually. Total lie. Absolutely does not as it show in ad.",
  "repliedAt": null,
  "replyContent": null,
  "reviewCreatedVersion": null,
  "reviewId": "14615460-bfb9-4b1e-a449-cd07a4c78b09",
  "score": 1,
  "sortOrder": "newest",
  "thumbsUpCount": 2,
  "userImage": "https://play-lh.googleusercontent.com/a/ALm5wu3ZyrOhN3anWqdM1KeY_GeThMW5e9ZaEvCkPNRv=mo",
  "userName": "mit nat"
}



In [28]:
app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.to_csv('top_reviews.csv', index=None, header=True)